In [3]:
import pandas as pd

# Load the provided files to inspect their content

customers_path = r'C:\Users\akash\Downloads\Customers.csv'
transactions_path = r'C:\Users\akash\Downloads\Transactions.csv'
products_path = r'C:\Users\akash\Downloads\Products.csv'

customers_df = pd.read_csv(customers_path)
products_df = pd.read_csv(products_path)
transactions_df = pd.read_csv(transactions_path)

# Display the first few rows of each file
customers_head = customers_df.head()
products_head = products_df.head()
transactions_head = transactions_df.head()

customers_head, products_head, transactions_head


(  CustomerID        CustomerName         Region  SignupDate
 0      C0001    Lawrence Carroll  South America  2022-07-10
 1      C0002      Elizabeth Lutz           Asia  2022-02-13
 2      C0003      Michael Rivera  South America  2024-03-07
 3      C0004  Kathleen Rodriguez  South America  2022-10-09
 4      C0005         Laura Weber           Asia  2022-08-15,
   ProductID              ProductName     Category   Price
 0      P001     ActiveWear Biography        Books  169.30
 1      P002    ActiveWear Smartwatch  Electronics  346.30
 2      P003  ComfortLiving Biography        Books   44.12
 3      P004            BookWorld Rug   Home Decor   95.69
 4      P005          TechPro T-Shirt     Clothing  429.31,
   TransactionID CustomerID ProductID      TransactionDate  Quantity  \
 0        T00001      C0199      P067  2024-08-25 12:38:23         1   
 1        T00112      C0146      P067  2024-05-27 22:23:54         1   
 2        T00166      C0127      P067  2024-04-25 07:38:55    

In [4]:
# Check for null values in all datasets to ensure completeness
customers_null = customers_df.isnull().sum()
products_null = products_df.isnull().sum()
transactions_null = transactions_df.isnull().sum()

# Check the number of unique CustomerIDs, ProductIDs in each dataset
unique_customers = customers_df['CustomerID'].nunique()
unique_products = products_df['ProductID'].nunique()
transactions_customers = transactions_df['CustomerID'].nunique()
transactions_products = transactions_df['ProductID'].nunique()

{
    "Customers_Null_Values": customers_null,
    "Products_Null_Values": products_null,
    "Transactions_Null_Values": transactions_null,
    "Unique_Customers_in_Customers": unique_customers,
    "Unique_Customers_in_Transactions": transactions_customers,
    "Unique_Products_in_Products": unique_products,
    "Unique_Products_in_Transactions": transactions_products,
}


{'Customers_Null_Values': CustomerID      0
 CustomerName    0
 Region          0
 SignupDate      0
 dtype: int64,
 'Products_Null_Values': ProductID      0
 ProductName    0
 Category       0
 Price          0
 dtype: int64,
 'Transactions_Null_Values': TransactionID      0
 CustomerID         0
 ProductID          0
 TransactionDate    0
 Quantity           0
 TotalValue         0
 Price              0
 dtype: int64,
 'Unique_Customers_in_Customers': 200,
 'Unique_Customers_in_Transactions': 199,
 'Unique_Products_in_Products': 100,
 'Unique_Products_in_Transactions': 100}

In [5]:
# Merge Transactions with Products to get product details for each transaction
transactions_products = pd.merge(transactions_df, products_df, on="ProductID", how="left")

# Merge the result with Customers to associate transactions with customer details
full_data = pd.merge(transactions_products, customers_df, on="CustomerID", how="left")

# Display the first few rows of the merged dataset
full_data.head()


,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,ProductName,Category,Price_y,CustomerName,Region,SignupDate
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Andrea Jenkins,Europe,2022-12-03
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Brittany Harvey,Asia,2024-09-04
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Kathryn Stevens,Europe,2024-04-04
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Travis Campbell,South America,2024-04-11
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Timothy Perez,Europe,2022-03-15


In [6]:
# Aggregate customer profiles based on transaction and product data

# Calculate spending per category and transaction stats
customer_profiles = (
    full_data.groupby(["CustomerID", "Region"])
    .agg(
        total_spending=("TotalValue", "sum"),
        transaction_count=("TransactionID", "count"),
        avg_transaction_value=("TotalValue", "mean"),
    )
    .reset_index()
)

# Add category spending as proportions of total spending
category_spending = (
    full_data.groupby(["CustomerID", "Category"])["TotalValue"].sum().unstack(fill_value=0)
)
category_spending = category_spending.div(category_spending.sum(axis=1), axis=0).reset_index()

# Merge the two datasets to complete customer profiles
customer_profiles = pd.merge(customer_profiles, category_spending, on="CustomerID", how="left")

# Display the first few rows of the customer profiles
customer_profiles.head()


,CustomerID,Region,total_spending,transaction_count,avg_transaction_value,Books,Clothing,Electronics,Home Decor
0,C0001,South America,3354.52,5,670.904,0.034163,0.000000,0.842833,0.123004
1,C0002,Asia,1862.74,4,465.685,0.000000,0.550512,0.000000,0.449488
2,C0003,South America,2725.38,4,681.345,0.000000,0.044896,0.508259,0.446844
3,C0004,South America,5354.88,8,669.360,0.352665,0.000000,0.253178,0.394156
4,C0005,Asia,2034.24,3,678.080,0.000000,0.000000,0.580256,0.419744
